In [2]:
from torchkf import *
import numpy as np

import plotly.express as px
import plotly.graph_objs as go

import symengine as si

In [73]:
# a = np.exp(6)
def pendulum_f(x,v,p): 
    q, w = x[:, 0]
    acc  = -p[0, 0] * q - 0.05 * w
    dx   = si.Matrix([[w], [acc]])
    return dx

models = [
    GaussianModel(
        fsymb=pendulum_f, 
        g=lambda x,v,p: x[0, None], 
        n=2, m=1, pE=np.array([9.81 * 0.1]), p=1, l=1, sv=1/8., sw=1/2.,
#         V=np.asarray([np.exp(6)]), 
        Q=[np.diag([np.exp(9)])],
        R=[np.diag([np.exp(32), np.exp(24)])], 
#         R=np.
        x=np.asarray([[np.pi/10.], [0]])
    )
]
hdm = HierarchicalGaussianModel(*models, dt=0.1)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.00s)
  Compiling g... g() ok. (compiled in 0.00s)
Done. 


In [74]:
gen = DEMInversion(hdm, states_embedding_order=8).generate(2048)
px.line(y=[gen.x[:, 0, 0, 0], gen.x[:, 0, 1, 0], gen.v[:, 0, 0, 0]])

  0%|          | 0/2048 [00:00<?, ?it/s]

In [5]:
# models = [
#     GaussianModel(
#         f=pendulum_f, 
#         g=lambda x,v,p: x[0, None], 
#         n=2, m=1, pE=np.array([9.81 * 0.1]), p=1, l=1, sv=1/16., sw=1/16.,
#         V=np.asarray([np.exp(32)]), 
#         W=np.asarray([np.exp(32), np.exp(-16)]), 
#         x=np.asarray([[np.pi/10.], [0]])
#     )
# ]
# hdm = HierarchicalGaussianModel(*models, dt=0.1)
# hdm[0].sw = 1/2.
# # hdm[0].sv = 1/2.
# # hdm.dt = 0.1
# z, _ = dem_z(hdm, 1024)
# def autocorr(x):
#     x = x.squeeze()
#     result = np.correlate(x, x, mode='full')
#     return result[int(result.size/2):][:20] / result[result.size//2]
# print(z[0].var())
# px.line(y=autocorr(z[0]))


In [6]:
px.imshow(DEMInversion.generalized_covariance(8, 1/2))

In [106]:
models = [
    GaussianModel(
        fsymb=pendulum_f, 
        g=lambda x,v,p: x[0, None], 
        n=2, m=1, pE=np.array([9.81 * 0.1]), p=1, l=1, sv=1/8., sw=1/2.,
        Q=[np.diag([np.exp(9)])],
        R=[np.diag([np.exp(16), np.exp(4)])], 
        x=np.asarray([[np.pi/10.], [0]])
    )
]
hdm = HierarchicalGaussianModel(*models, dt=0.1)

gen = DEMInversion(hdm, states_embedding_order=8).generate(2048)
x  =  gen.v[:, 0, :1, 0]
dec = DEMInversion(hdm, states_embedding_order=8).run(x, nE=1)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.00s)
  Compiling g... g() ok. (compiled in 0.00s)
Done. 


  0%|          | 0/2048 [00:00<?, ?it/s]

  M-step:   0%|          | 0/8 [00:00<?, ?it/s]

timestep:   0%|          | 0/2048 [00:00<?, ?it/s]

In [107]:
emb = DEMInversion.generalized_coordinates(dec.qU.x[:, 0, :1], p=10, dt=0.1)
px.line(y=[dec.qU.x[:, 7, 0], dec.qU.x[:, 6, 1]])#, dec.qU.x[:, 1, 0], emb[:, 1, 0]])

In [9]:
n   = 5
nxe = dec.qU.x.shape[1] * dec.qU.x.shape[2]
nx  = int(nxe / 5)
nve = dec.qU.y.shape[1] * dec.qU.y.shape[2]
nv  = int(nve / 5)

Cxe = dec.qU.c[:, :nxe, :nxe]
Cxe = Cxe.reshape((Cxe.shape[0], n, nx, n, nx))
Cx  = np.einsum('tnini->tni', Cxe)

Cve = dec.qU.c[:, nxe:nxe+nve, nxe:nxe+nve]
Cve = Cve.reshape((Cve.shape[0], n, nv, n, nv))
Cv  = np.einsum('tnini->tni', Cve)

# for i in range(nx):
#     px.imshow(Cxe[128,:,i,:,i]).show()
# for i in range(nv):
#     px.imshow(Cve[0,:,i,:,i]).show()

ValueError: cannot reshape array of size 663552 into shape (2048,5,3,5,3)

In [ ]:
emb = DEMInversion.generalized_coordinates(dec.qU.x[:, 0, :1], p=2, dt=0.1)
px.line(y=[dec.qU.x[:, 0, 0], dec.qU.x[:, 0, 1], dec.qU.x[:, 1, 0], emb[:, 1, 0]])

In [37]:
def high_order_pendulum(n): 
    def pendulum_f(x,v,p):
        D        = np.zeros((n, n), dtype='object')
        D[:-1]   = np.eye(n-1, n, 1)        
        D[1:, :] = -p[0, 0]
        dx       = (D @ x).reshape((-1, 1)) - 0.001*  x[:-1].sum()
        dx       = si.Matrix(dx.tolist())
        return dx
    return pendulum_f

n = 4
models = [
    GaussianModel(
        fsymb=high_order_pendulum(n), 
        g=lambda x,v,p: x[0, None], 
        n=n, m=1, pE=np.array([9.81 * 0.1]), p=1, l=1, sv=1/8., sw=1/2.,
#         V=np.asarray([np.exp(6)]), 
        Q=[np.diag([np.exp(9)])],
        R=[np.diag([np.exp(32)]*(n-1)+ [np.exp(2)])], 
#         R=np.
        x=np.asarray([np.pi/10.]+ [0]*(n-1)).reshape((-1, 1))
    )
]
hdm = HierarchicalGaussianModel(*models, dt=0.1)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.00s)
  Compiling g... g() ok. (compiled in 0.00s)
Done. 


In [39]:
gen = DEMInversion(hdm, states_embedding_order=8).generate(2048)
px.line(y=[gen.x[:, 7, 0, 0], gen.x[:, 6, 1, 0], gen.v[:, 0, 0, 0]])

  0%|          | 0/2048 [00:00<?, ?it/s]